# Project "Advanced Lane Finding"

In [ ]:
import alf
import glob
import imp
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 
%matplotlib inline

FORCE_CALIBRATION = True
CALIB_DATA_FILE_NAME = "./cam_calib_data.p"

imp.reload(alf)

fileList = glob.glob("./camera_cal/calibration*.jpg")
mtx, dist = alf.getCameraCalibrationMatrix(fileList, CALIB_DATA_FILE_NAME, FORCE_CALIBRATION)


In [ ]:
import cv2
import os

#IMAGE_FILE = "test_images/straight_lines1.jpg"
IMAGE_FILE = "test_images/test3.jpg"
#IMAGE_FILE = "test_images/project_video_1039.jpg"
#IMAGE_FILE = "test_images/project_video_1151.jpg"
#IMAGE_FILE = "test_images/project_video_559.jpg"
#IMAGE_FILE = "test_images/challenge_video_133.jpg"


# Load the image
img = mpimg.imread(IMAGE_FILE)

# Undistort the image
imgUndist = alf.undistortImage(img, mtx, dist)

# Save the undistorted image
#mpimg.imsave("./undist_images/"+os.path.basename(IMAGE_FILE), imgUndist)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title("Original image")
ax2.imshow(imgUndist)
ax2.set_title("Undistorted image")


In [ ]:
CROP_Y          = 453
WARP_X_BOTTOM   = 150
WARP_X_TOP_LEFT = 586

h, w = imgUndist.shape[:2]

# Crop the upper part of the image 
imgCrop = imgUndist[CROP_Y:h, :]
        
warpXTopRight = imgCrop.shape[1]-WARP_X_TOP_LEFT
srcRect = ((WARP_X_BOTTOM,h), (w-WARP_X_BOTTOM,h), (warpXTopRight, CROP_Y), (WARP_X_TOP_LEFT, CROP_Y))

print("imgCrop.shape:", imgCrop.shape)
hCrop = imgCrop.shape[0]
srcRect2 = ((WARP_X_BOTTOM,hCrop), (w-WARP_X_BOTTOM,hCrop), (warpXTopRight, 0), (WARP_X_TOP_LEFT, 0))

if False:
    plt.imshow(imgCrop)
    for i in range(1,len(srcRect2)):
        j = (i+1) % len(srcRect2)
        plt.plot([srcRect2[j][0], srcRect2[i][0]], [srcRect2[j][1], srcRect2[i][1]], 'r-')
else:
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
    ax1.imshow(imgUndist)
    ax1.set_title("Undistorted image")
    for i in range(1,len(srcRect)):
        j = (i+1) % len(srcRect)
        ax1.plot([srcRect[j][0], srcRect[i][0]], [srcRect[j][1], srcRect[i][1]], 'r-')

    ax2.imshow(imgCrop)
    ax2.set_title("Cropped image")
    for i in range(1,len(srcRect)):
        j = (i+1) % len(srcRect)
        ax2.plot([srcRect2[j][0], srcRect2[i][0]], [srcRect2[j][1], srcRect2[i][1]], 'r-')



In [ ]:
##imgHlsW = cv2.cvtColor(imgCrop, cv2.COLOR_RGB2CMY)
## Convert to color space CMY
#imgHlsW = np.zeros_like(imgCrop)
#for plane in range(3):
#    imgHlsW[:,:,plane] = 255-imgCrop[:,:,plane]
#
#
#imgLS = np.zeros_like(imgHlsW[:,:,2])
#Y, X, _ = imgHlsW.shape
#for x in range(X):
#    for y in range(Y):
#        imgLS[y, x] = (float(imgHlsW[y, x, 1]) * float(imgHlsW[y, x, 2]))/256.
#
#fig, axs = plt.subplots(nrows=2, ncols=imgHlsW.shape[2], figsize=(20,5))
#
#planeNames = ['R', 'G', 'B']
#for i in range(imgHlsW.shape[2]):
#    axs[0,i].imshow(imgCrop[:,:,i], cmap="gray")
#    axs[0,i].set_title(planeNames[i])
#
#
#planeNames = ['C', 'M', 'Y']
#for i in range(imgHlsW.shape[2]):
#    axs[1,i].imshow(imgHlsW[:,:,i], cmap="gray")
#    axs[1,i].set_title(planeNames[i])


In [ ]:
# Warp the image to (nearly) birds view

dstRect = ((WARP_X_BOTTOM,h), (w-WARP_X_BOTTOM,h), (w-WARP_X_BOTTOM, 0), (WARP_X_BOTTOM, 0))
dstRect2 = ((WARP_X_BOTTOM,hCrop), (w-WARP_X_BOTTOM,hCrop), (w-WARP_X_BOTTOM, 0), (WARP_X_BOTTOM, 0))

print("srcRect:", srcRect)
print("dstRect:", dstRect)

print("srcRect2:", srcRect2)
print("dstRect2:", dstRect2)

Mwarp   = cv2.getPerspectiveTransform(np.float32(srcRect2), np.float32(dstRect))
Munwarp = cv2.getPerspectiveTransform(np.float32(dstRect),  np.float32(srcRect2))

imgWarped = cv2.warpPerspective(imgCrop, Mwarp, (w,h), flags=cv2.INTER_LINEAR)
print("imgWarped.shape:", imgWarped.shape)

Mwarp2   = cv2.getPerspectiveTransform(np.float32(srcRect2), np.float32(dstRect2))
Munwarp2 = cv2.getPerspectiveTransform(np.float32(dstRect2), np.float32(srcRect2))
imgWarped2 = cv2.warpPerspective(imgCrop, Mwarp2, (w,hCrop), flags=cv2.INTER_LINEAR)
print("imgCrop.shape:", imgCrop.shape)
print("imgWarped2.shape:", imgWarped2.shape)

f, sub = plt.subplots(1, 3, figsize=(20,10))
sub[0].imshow(imgWarped)
sub[0].set_title("imgWarped")
sub[1].imshow(imgWarped2)
sub[1].set_title("imgWarped2")
sub[2].imshow(cv2.warpPerspective(imgWarped, Munwarp, (w,h), flags=cv2.INTER_LINEAR))
sub[2].set_title("imgUnwarped")


In [ ]:
## Convert the color space of warped image
#
#imgProc = imgWarped
##imgHlsW = cv2.cvtColor(imgProc, cv2.COLOR_RGB2HLS)
#imgHlsW = np.zeros_like(imgProc)
#planeNames = ['C', 'M', 'Y']
#for plane in range(3):
#    imgHlsW[:,:,plane] = 255-imgProc[:,:,plane]
#
##imgLS = np.matmul(imgHlsW[:,:,1], imgHlsW[:,:,2])
#
#imgLS = np.zeros_like(imgHlsW[:,:,2])
#Y, X, _ = imgHlsW.shape
#for x in range(X):
#    for y in range(Y):
#        imgLS[y, x] = (float(imgHlsW[y, x, 1]) * float(imgHlsW[y, x, 2]))/256.
#
#fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(hCrop//10,w//10))
#
#for i in range(imgHlsW.shape[2]):
#    axs[i].imshow(imgWarped[:,:,i], cmap="gray")
#    axs[i].set_title(planeNames[i])

In [ ]:
imgProc = imgWarped


# Remove dark parts of the image
imgBright = np.copy(imgProc[:,:,0])
lowLimit = int(imgBright.mean() * 1.3)
np.putmask(imgBright, imgBright<lowLimit, lowLimit)
plt.imshow(imgBright, cmap="gray", vmin=0, vmax=255)

In [ ]:
# Apply Sobel operator to saturation plane of bird eyes view 

magThresh = (10, 250)
imgSat = imgBright
#imgSat = imgWarped[:,:,0]
#imgSat = imgHlsW[:,:,1]
#imgSat = imgLS

imgSobelAbs = np.abs(cv2.Sobel(imgSat, cv2.CV_64F, 1, 0))
imgSobelScale = np.uint8(255.0*imgSobelAbs/imgSobelAbs.max())
imgSobelBin = np.zeros_like(imgSat) # Remove this line
imgSobelBin[(magThresh[0] < imgSobelScale) & (imgSobelScale < magThresh[1])] = 1


plt.imshow(imgSobelBin, cmap='gray')
plt.title("imgSobelBin")

In [ ]:
histogram = np.sum(imgSobelBin[imgSobelBin.shape[0]//2:,:], axis=0)

smoothWin = np.ones(51)
hist_smooth = np.convolve(smoothWin/smoothWin.sum(), histogram, 'same')

midpoint = histogram.shape[0]//2
leftx_base = np.argmax(hist_smooth[:midpoint])
rightx_base = np.argmax(hist_smooth[midpoint:]) + midpoint
print("1) left:", leftx_base, "  right:", rightx_base)

x_base = np.argmax(hist_smooth)
if x_base < midpoint:
    leftx_base = x_base
    search_from = x_base+500
    print("leftx_base found -> search to the right from", search_from)
    rightx_base = np.argmax(hist_smooth[search_from:]) + search_from
else:
    rightx_base = x_base
    search_to = rightx_base-500
    print("rightx_base found -> search to the left from", search_to)
    leftx_base = np.argmax(hist_smooth[:search_to])
print("2) left:", leftx_base, "  right:", rightx_base)

fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

axs[0].set_title("histogram")
axs[0].plot(histogram)
axs[1].set_title("hist_smooth")
axs[1].plot(hist_smooth)

In [ ]:
NUM_OF_HIST_WINDOWS=9

out_img = np.dstack((imgSobelBin, imgSobelBin, imgSobelBin))*255

if True:
    histogram = np.sum(imgSobelBin[imgSobelBin.shape[0]//2:,:], axis=0)

    # Smooth the histogram
    smoothWin = np.ones(51)
    histogram = np.convolve(smoothWin/smoothWin.sum(), histogram, 'same')


    midpoint = histogram.shape[0]//2
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    print("left:", leftx_base, "  right:", rightx_base)

    window_height = imgSobelBin.shape[0]//NUM_OF_HIST_WINDOWS
    nonzero = imgSobelBin.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    leftx_current = leftx_base
    rightx_current = rightx_base
    margin = 100
    minpix = 50
    left_lane_inds = []
    right_lane_inds = []

    for window in range(NUM_OF_HIST_WINDOWS):
        win_y_bottom = imgSobelBin.shape[0] - (window+1)*window_height
        win_y_top = imgSobelBin.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_bottom),(win_xleft_high,win_y_top),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_bottom),(win_xright_high,win_y_top),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_bottom) & (nonzeroy < win_y_top) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_bottom) & (nonzeroy < win_y_top) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 
else:
    print("imgSobelBin.shape:", imgSobelBin.shape)
    leftx, lefty, rightx, righty, nonzerox, nonzeroy, left_lane_inds, right_lane_inds = alf.searchLanes(imgSobelBin, NUM_OF_HIST_WINDOWS)

print("len(leftx) :", len(leftx))
print("len(rightx):", len(rightx))


# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)

print("left_fit:", left_fit, "   right_fit:", right_fit)
#plt.plot(histogram)
#plt.title("histogram")
# Generate x and y values for plotting
ploty = np.linspace(0, imgSobelBin.shape[0]-1, imgSobelBin.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 255, 255]
plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow', linewidth=3)
plt.plot(right_fitx, ploty, color='yellow', linewidth=3)


In [ ]:
ym_per_pixel = 15/imgSobelBin.shape[0]
xm_per_pixel = 3.7/700

y_eval = np.max(ploty)

left_fit_cr = np.polyfit(lefty * ym_per_pixel, leftx * xm_per_pixel, 2)
right_fit_cr = np.polyfit(righty * ym_per_pixel, rightx * xm_per_pixel, 2)

# Calculate curvature
left_curverad  = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pixel + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pixel + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

print("left radius:", left_curverad, "m,  right radius:", right_curverad, "m")

lane_pos_y = imgSobelBin.shape[0]-1
left_lane_pos = left_fit[0]*lane_pos_y**2 + left_fit[1]*lane_pos_y + left_fit[2]
right_lane_pos = right_fit[0]*lane_pos_y**2 + right_fit[1]*lane_pos_y + right_fit[2]

#print("lane pos left:", left_lane_pos, "  right:", right_lane_pos)
#print("lane pos type(left):", type(left_lane_pos), "  right:", right_lane_pos)

if len(leftx) > len(rightx):
    curveRadius = left_curverad
else:
    curveRadius = right_curverad
print()

lane_center = (left_lane_pos + right_lane_pos) / 2
dist_from_lane_center = lane_center - imgSobelBin.shape[1]//2
abs_dist_in_meters = np.abs(dist_from_lane_center * xm_per_pixel)
if dist_from_lane_center < 0:
    position = "right"
else:
    position = "left"

textRadius   = "Radius of curvature: {:8f}m".format(curveRadius)
textPosition = "Vehicle is {:5.2f}m {:s} of center".format(abs_dist_in_meters, position)
print(textRadius)
print(textPosition)

In [ ]:
overlayOffs = imgUndist.shape[0] - imgCrop.shape[0]

imgMarked = np.zeros_like(imgUndist)
imgMarked[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
imgMarked[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 255, 255]
if True:
    for i in range(len(left_fitx)):
        y = int(ploty[i])
    #    cv2.circle(imgMarked, (int(left_fitx[i]), y), 5, (255,0,0))
    #    cv2.circle(imgMarked, (int(right_fitx[i]), y), 5, (0, 255,255))
        cv2.line(imgMarked, (int(left_fitx[i]), y), (int(right_fitx[i]), y), color=(0, 255,0))
else:
    #xx = np.concatenate((left_fitx, right_fitx[-1:0:-1]))
    #print("xx:", xx)
    #yy = np.zeros_like(xx)
    #print("yy:", yy)
    #
    #yy.fill(y)
    #print("yy:", yy)
    
    #xy = zip(xx,yy)
    xy = np.zeros((len(left_fitx) + len(right_fitx), 2), dtype=np.int)
    for i in range(len(left_fitx)):
        xy[i] = (left_fitx[i], y)

    for i in range(len(right_fitx)-1,-1,-1):
        xy[i+len(left_fitx)] = (right_fitx[i], y)

    #xy = np.array([[100,100], [300,100], [300,200], [100,200]], np.int32)
    xy = xy.reshape((-1,1,2))
    print("xy:", xy)
    cv2.fillConvexPoly(imgMarked, xy, (255,255,255))

imgUnwarped = np.zeros_like(imgMarked)
imgUnwarped[:,:] = cv2.warpPerspective(imgMarked, Munwarp, (w, h), flags=cv2.INTER_LINEAR)

imgOverlayed = np.copy(imgUndist)

imgUnwarpedShifted = np.zeros_like(imgUnwarped)
imgUnwarpedShifted[overlayOffs:,:] = imgUnwarped[:imgCrop.shape[0],:]
imgOverlayed = cv2.addWeighted(imgUndist, 1, imgUnwarpedShifted, 0.2, 0)
cv2.putText(imgOverlayed, textRadius,   (150, 50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255, 255, 255))
cv2.putText(imgOverlayed, textPosition, (150,100), cv2.FONT_HERSHEY_COMPLEX_SMALL, 2, (255, 255, 255))

fig, sub = plt.subplots(1, 3, figsize=(20,10))
sub[0].imshow(imgMarked)
sub[0].set_title("warped")
sub[1].imshow(imgUnwarped)
sub[1].set_title("unwarped")
sub[2].imshow(imgOverlayed)
sub[2].set_title("overlayed")
